# Caso de uso de IA generativa: resumir o diálogo

Bem-vindo ao lado prático deste curso. Neste laboratório você realizará a tarefa de resumo de diálogos usando IA generativa. Você explorará como o texto de entrada afeta a saída do modelo e realizará engenharia imediata para direcioná-lo para a tarefa necessária. Ao comparar inferências de disparo zero, de disparo único e de poucos disparos, você dará o primeiro passo em direção à engenharia imediata e verá como ela pode aprimorar a produção generativa de modelos de linguagem grande.

# Índice

- [1 - Configurar Kernel e Dependências Necessárias](#1)
- [ 2 - Resuma o diálogo sem solicitar engenharia](#2)
- [3 - Resuma o diálogo com um prompt de instrução](#3)
- [3.1 - Inferência Zero Shot com um prompt de instrução](#3.1)
- [3.2 - Inferência Zero Shot com o modelo de prompt do FLAN-T5](#3.2)
- [ 4 - Resuma o diálogo com inferência de um tiro e poucos tiros](#4)
- [4.1 - Inferência One Shot](#4.1)
- [4.2 - Inferência de poucos tiros](#4.2)
- [ 5 - Parâmetros de configuração generativa para inferência](#5)

<a nome='1'></a>
## 1 - Configurar o kernel e as dependências necessárias

In [ ]:
!pip install torch torchdata transformers datasets

Carregue os conjuntos de dados, Large Language Model (LLM), tokenizer e configurador. Não se preocupe se você ainda não entendeu todos esses componentes - eles serão descritos e discutidos posteriormente neste caderno.

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

<a nome='2'></a>
## 2 - Resuma o diálogo sem solicitar engenharia

Neste caso de uso, você gerará um resumo de um diálogo com o Large Language Model (LLM) FLAN-T5 pré-treinado do Hugging Face. A lista de modelos disponíveis no pacote Hugging Face `transformers` pode ser encontrada [aqui](https://huggingface.co/docs/transformers/index).

Vamos fazer upload de alguns diálogos simples do conjunto de dados [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) Hugging Face. Este conjunto de dados contém mais de 10.000 diálogos com os resumos e tópicos rotulados manualmente correspondentes.

In [4]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Imprima alguns diálogos com seus resumos básicos.

In [5]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exa

Carregue o [modelo FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5), criando uma instância da classe `AutoModelForSeq2SeqLM` com o método `.from_pretrained()`.

In [ ]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Para realizar codificação e decodificação, você precisa trabalhar com texto em formato tokenizado. **Tokenização** é o processo de divisão de textos em unidades menores que podem ser processadas pelos modelos LLM.

Baixe o tokenizer para o modelo FLAN-T5 usando o método `AutoTokenizer.from_pretrained()`. O parâmetro `use_fast` ativa o tokenizer rápido. Neste estágio, não há necessidade de entrar em detalhes sobre isso, mas você pode encontrar os parâmetros do tokenizer na [documentação](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/auto #transformers.AutoTokenizer).

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Teste a codificação do tokenizer e a decodificação de uma frase simples:

In [8]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

DECODED SENTENCE:
What time is it, Tom?


Agora é hora de explorar quão bem o LLM básico resume um diálogo sem qualquer engenharia imediata. **Engenharia de prompt** é um ato humano de alterar o **prompt** (entrada) para melhorar a resposta de uma determinada tarefa.

In [9]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

-------------------------------

Você pode ver que as suposições do modelo fazem algum sentido, mas não parece ter certeza de qual tarefa ele deve realizar. Parece que apenas constitui a próxima frase do diálogo. A engenharia imediata pode ajudar aqui.

<a nome='3'></a>
## 3 - Resuma o diálogo com um prompt de instruções

A engenharia de prompt é um conceito importante no uso de modelos básicos para geração de texto. Você pode conferir [este blog](https://www.amazon.science/blog/emnlp-prompt-engineering-is-the-new-feature-engineering) da Amazon Science para obter uma introdução rápida à engenharia imediata.

<a nome='3.1'></a>
### 3.1 - Inferência Zero Shot com um prompt de instrução

Para instruir o modelo a executar uma tarefa – resumir um diálogo – você pode pegar o diálogo e convertê-lo em um prompt de instrução. Isso geralmente é chamado de **inferência de disparo zero**. Você pode conferir [este blog da AWS](https://aws.amazon.com/blogs/machine-learning/zero-shot-prompting-for-the-flan-t5-foundation-model-in-amazon-sagemaker -jumpstart/) para uma rápida descrição do que é o aprendizado zero shot e por que ele é um conceito importante para o modelo LLM.

Envolva o diálogo em uma instrução descritiva e veja como o texto gerado mudará:

In [11]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Provide a one-line summary of the following conversation.

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Provide a one-line summary of the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
T

Isto é muito melhor! Mas o modelo ainda não capta as nuances das conversas.

**Exercício:**

- Experimente o texto `prompt` e veja como as inferências serão alteradas. As inferências mudarão se você encerrar o prompt apenas com uma string vazia em vez de `Resumo: `?
- Tente reformular o início do texto do `prompt` de `Resuma a seguinte conversa.` para algo diferente - e veja como isso influenciará a saída gerada.

<a nome='3.2'></a>
### 3.2 - Inferência Zero Shot com o modelo de prompt do FLAN-T5

Vamos usar um prompt um pouco diferente. O FLAN-T5 tem muitos modelos de prompt publicados para determinadas tarefas [aqui](https://github.com/google-research/FLAN/tree/main/flan/v2). No código a seguir, você usará um dos [prompts FLAN-T5 pré-construídos](https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py):

In [12]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Tom is late for the train.

--------------

Observe que esse prompt do FLAN-T5 ajudou um pouco, mas ainda tem dificuldade para captar as nuances da conversa. Isso é o que você tentará resolver com algumas inferências.

<a nome='4'></a>
## 4 - Resuma o diálogo com inferência de um tiro e de poucos tiros

**Inferência única e de poucas tentativas** são as práticas de fornecer a um LLM um ou mais exemplos completos de pares de prompt-resposta que correspondem à sua tarefa - antes do prompt real que você deseja concluir. Isso é chamado de “aprendizado em contexto” e coloca seu modelo em um estado que compreende sua tarefa específica. Você pode ler mais sobre isso [neste blog do HuggingFace](https://huggingface.co/blog/few-shot-learning-gpt-neo-and-inference-api).

<a nome='4.1'></a>
### 4.1 - Inferência One Shot

Vamos construir uma função que pega uma lista de `example_indices_full`, gera um prompt com exemplos completos e, no final, anexa o prompt que você deseja que o modelo complete (`example_index_to_summarize`). Você usará o mesmo modelo de prompt FLAN-T5 da seção [3.2](#3.2).

In [15]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""

    return prompt

Construa o prompt para realizar inferência única:

In [16]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also ne

Agora passe este prompt para realizar a inferência única:

In [17]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


<a nome='4.2'></a>
### 4.2 - Inferência de poucos tiros

Vamos explorar a inferência de algumas cenas adicionando mais dois pares completos de resumo de diálogo ao seu prompt.

In [18]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. 

Agora passe este prompt para realizar algumas inferências:

In [19]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


Neste caso, poucos disparos não proporcionaram muita melhoria em relação à inferência de um disparo. E qualquer coisa acima de 5 ou 6 doses normalmente também não ajuda muito. Além disso, você precisa ter certeza de não exceder o comprimento do contexto de entrada do modelo que, em nosso caso, é de 512 tokens. Qualquer coisa acima do comprimento do contexto será ignorada.

No entanto, você pode ver que fornecer pelo menos um exemplo completo (one shot) fornece ao modelo mais informações e melhora qualitativamente o resumo geral.

**Exercício:**

Experimente algumas inferências de tiro.
- Escolha diferentes diálogos - altere os índices na lista `example_indices_full` e o valor `example_index_to_summarize`.
- Altere o número de tiros. No entanto, certifique-se de permanecer dentro do comprimento de contexto 512 do modelo.

Quão bem a inferência de poucas tomadas funciona com outros exemplos?

<a nome='5'></a>
## 5 - Parâmetros de configuração generativos para inferência

Você pode alterar os parâmetros de configuração do método `generate()` para ver uma saída diferente do LLM. Até agora o único parâmetro que você configurou foi `max_new_tokens=50`, que define o número máximo de tokens a serem gerados. Uma lista completa de parâmetros disponíveis pode ser encontrada na [documentação do Hugging Face Generation](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationConfig).

Uma maneira conveniente de organizar os parâmetros de configuração é usar a classe `GenerationConfig`.

**Exercício:**

Altere os parâmetros de configuração para investigar sua influência na saída.

Colocando o parâmetro `do_sample = True`, você ativa várias estratégias de decodificação que influenciam o próximo token da distribuição de probabilidade em todo o vocabulário. Você pode então ajustar as saídas alterando a `temperatura` e outros parâmetros (como `top_k` e `top_p`).

Remova o comentário das linhas da célula abaixo e execute novamente o código. Tente analisar os resultados. Você pode ler alguns comentários abaixo.

In [20]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=20)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Many people try to upgrade their system in order to make more sales.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.



Comentários relacionados à escolha dos parâmetros na célula de código acima:
- Escolher `max_new_tokens=10` tornará o texto de saída muito curto, então o resumo do diálogo será cortado.
- Colocando `do_sample = True` e alterando o valor da temperatura você obtém mais flexibilidade na saída.

Como você pode ver, a engenharia imediata pode ser muito útil nesse caso de uso, mas existem algumas limitações. A seguir, você começará a explorar como pode usar o ajuste fino para ajudar seu LLM a entender um caso de uso específico com mais profundidade!